# Problem 1

The main point of this problem is to estimate how far away our sun sits from the center of the Milky Way, emulating the work of Shapley.

We begin by importing useful packages, such as pandas (for data manipulation), numpy(for math), and matplotlib.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

We are given a CSV with information on globular clusters within the halo of the Milky Way. We need to read in the data, and examine it. Immediately, we can see that our right ascention and declination columns need to be converted into decimal equivalents.

In [29]:
data=pd.read_csv('C:\\Users\\lazyp\\Desktop\\Willamette Univeristy\\SPRING 2024\\data275\\homework-4-kendall-and-sam-main\\data\\MW_star_clusters.csv')
data

,Identifier,Right ascension,Declination,Constellation,Distance (kpc)
0,FSR 9,18h 28m 30.6s,-31° 54′ 24″,Sagittarius,6.9
1,FSR 19,17h 35m 38.4s,-21° 04′ 12″,Ophiuchus,7.2
2,FSR 25,17h 41m 43.2s,-19° 34′ 16″,Ophiuchus,7.0
3,FSR 190,20h 05m 31.3s,+33° 34′ 09″,Cygnus,10.0
4,FSR 584,02h 27m 15s,+61° 37′ 28″,Cassiopeia,1.4
...,...,...,...,...,...
249,VVV CL150,17h 50m 41.0s,-25° 13′ 06″,Sagittarius,8.1
250,VVV CL153,17h 53m 32.0s,-25° 22′ 56″,Sagittarius,10.0
251,VVV CL154,17h 55m 08.0s,-28° 06′ 01″,Sagittarius,8.9
252,VVV CL160,18h 06m 57.0s,-20° 00′ 40″,Sagittarius,6.8


We convert these columns by doing some string manipulation, which we save as functions. We converted the Right Ascension from hours into decimals via the formula $((\text{hours})+\frac{\text{minutes}}{60}+\frac{\text{seconds}}{3600})*15$. For the declination we just used the formula $(\text{degrees}+\frac{\text{minutes}}{60}+\frac{\text{seconds}}{3600})$. 

In [30]:
def convert_to_RA(strng):
    strng=strng
    strng=strng.replace('h','')
    strng=strng.replace('m','')
    strng=strng.replace('s','')
    strng=strng.rstrip()
    strng=re.sub(r'\n.*','',strng,flags=re.DOTALL)
    strng=strng.split()
    strng=(float(strng[0])+(float(strng[1])/60)+(float(strng[2])/3600))*15
    return strng

def convert_to_DEC(strng):
    strng=strng
    strng=strng.replace('°','')
    strng=strng.replace('′','')
    strng=strng.replace('″','')
    strng=strng.rstrip()
    strng=re.sub(r'\n.*','',strng,flags=re.DOTALL)
    strng=strng.split()
    strng=float(strng[0])+(float(strng[1])/60)+(float(strng[2])/3600)
    return strng

We then apply these functions to our data:

In [31]:
data['RA_deg']=data['Right ascension'].apply(convert_to_RA)
data['DEC_deg']=data['Declination'].apply(convert_to_DEC)


By using these formulas  we can calculate the x,y and z distance from us using Right Ascension($\delta$), Declenation($\alpha$) and distance to star($R$). 

Formula to find X cordinate:
$X = R \cos(\delta)\cos(\alpha)$

Formula to find Y cordinate:
$Y = R \cos(\delta)\sin(\alpha)$ 

Formula to find Z cordinate:
$Z = R \sin(\delta)$ 

For ease of use, we have created functions.

In [32]:
def convert_distance_to_x(distance,Declination,Right_ascension):
    return distance*np.cos(Declination*(np.pi/180))*np.cos(Right_ascension*(np.pi/180))
def convert_distance_to_y(distance,Declination,Right_ascension):
    return distance*np.cos(Declination*(np.pi/180))*np.sin(Right_ascension*(np.pi/180))
def convert_distance_to_z(distance,Declination):
    return distance*np.sin(Declination*(np.pi/180))

We then apply these function, and save the result as X_cord, Y_cord, and Z_cord columns in the data set.

In [33]:
data['X_cord']=convert_distance_to_x(data['Distance (kpc)'],data['DEC_deg'],data['RA_deg'])
data['Y_cord']=convert_distance_to_y(data['Distance (kpc)'],data['DEC_deg'],data['RA_deg'])
data['Z_cord']=convert_distance_to_z(data['Distance (kpc)'],data['DEC_deg'])

Once we have calculated the x,y and z cordinates of all the stars, we can then average all of them together to find the most average x cordinate, y cordinate and z cordinate.  We save these averages as x_mean, y_mean, and z_mean. These means will roughly tell us where the center of the galaxy is in reference to us.

In [34]:
x_mean=data['X_cord'].mean()
y_mean=data['Y_cord'].mean()
z_mean=data['Z_cord'].mean()

     
We then can use the average x,y and z cordinate to find how far we are from the center of the galaxy. This is done by using the distance formula $\sqrt{x^2 + y^2 + z^2}$. With this, we learn that we are 7.016 kiloparsecs away from the center of the Milky Way. This means we are about 1000 parsecs off of the actual number.
    


In [35]:
distance=np.sqrt(x_mean**2+y_mean**2+z_mean**2)
print(distance)

7.016525499501197


### Extra Credit

##### Constellation

For the extra credit problem, our goal is to find the constellation that is closest to the center of the Milky Way. In order to do this, we first define a function which will tell us how far a star is from the center of the Milky Way, and apply it to our data.

In [36]:
def distance_from_center(x_cord,y_cord,z_cord):
    return (x_mean-x_cord)**2+(y_mean-y_cord)**2+(z_mean-z_cord)**2
data['distance_from_center']=distance_from_center(data['X_cord'],data['Y_cord'],data['Z_cord'])

Now since there are multiple stars in each constellation, all of which are recorded in this dataset, we need to average out the distance of the stars within each constellation. We do this by creating a list of the unique constellations in the data set and a data frame to store our processed data.

In [45]:
constellation_list = data['Constellation'].unique()
star_frame = pd.DataFrame(constellation_list)

We initialize three empty lists: x_list, y_list, and z_list. These lists will be used to store the average distances of stars in each constellation along the x, y, and z axes, respectively.

In [46]:
x_list=[]
y_list=[]
z_list=[]

We iterate through each unique constellation name to calculate the average distances for stars within each constellation. Within the loop, we compute the average x, y, and z distances of stars belonging to the current constellation. This is achieved by filtering the dataset based on the current constellation and then calculating the mean values of the x, y, and z coordinates. We append the calculated average distances to their respective lists (x_list, y_list, and z_list) for storage.

In [47]:
for i in range(len(constilation_list)):
   x_distance=data['X_cord'][data['Constellation']==constilation_list[i]].mean()
   y_distance=data['Y_cord'][data['Constellation']==constilation_list[i]].mean()
   z_distance=data['Z_cord'][data['Constellation']==constilation_list[i]].mean()
   x_list.append(x_distance)
   y_list.append(y_distance)
   z_list.append(z_distance)

Finally, we add three new columns to the star_frame DataFrame to store the calculated average distances along the x, y, and z axes for each constellation.

In [48]:
star_frame['x_distance(avg)']=x_list
star_frame['y_distance(avg)']=y_list
star_frame['z_distance(avg)']=z_list

Now we need to calculate the distance from the center, which we can do with our previously defined function.

In [49]:
star_frame['distance to center']=distance_from_center(star_frame['x_distance(avg)'],star_frame['y_distance(avg)'],star_frame['z_distance(avg)'])

Finally, we need to identify which star has the shortest distance to the center of the Milky Way. We do this by indexing for the smallest 'distance to center' value.

In [50]:
closest_constellation_index = star_frame['distance to center'].argmin()


We then use that value to index out the actual star, which in this case is Scorpius.

In [51]:
closest_constellation_name = star_frame.iloc[closest_constellation_index, 0]
closest_constellation_name 

'Scorpius'

##### Center of the Milky Way in equatorial coordinates

In order to pin down the center of the Milky Way in equatorial coordinates, we first need the mean of the X, Y and Z coordinates.

In [56]:
x_mean = data['X_cord'].mean()
y_mean = data['Y_cord'].mean()
z_mean = data['Z_cord'].mean()

After that, we can work backwards to get degrees for our right ascention and declination. 

In [57]:
right_ascension = np.arctan2(y_mean, x_mean) * (180 / np.pi)  
if right_ascension < 0:
    right_ascension += 360 
declination = np.arcsin(z_mean / np.sqrt(x_mean**2 + y_mean**2 + z_mean**2)) * (180 / np.pi)


According to our calculations, these are the equatorial coordinates of the Milky Way :
Right Ascension: 253.98541899701445 degrees 
Declination: -30.895185851877127 degrees